# Module 1

### Query 1.1 
Import the csv file of the stock of your choosing using 'pd.read_csv()' function into a dataframe.
Shares of a company can be offered in more than one category. The category of a stock is indicated in the ‘Series’ column. If the csv file has data on more than one category, the ‘Date’ column will have repeating values. To avoid repetitions in the date, remove all the rows where 'Series' column is NOT 'EQ'.
Analyze and understand each column properly.
You'd find the head(), tail() and describe() functions to be immensely useful for exploration. You're free to carry out any other exploration of your own.

In [2]:
import pandas as pd
infosys = pd.read_csv('INFY.csv')
infosys = infosys[infosys.Series == 'EQ']

### Query 1.2
Calculate the maximum, minimum and mean price for the last 90 days. (price=Closing Price unless stated otherwise)

In [3]:
maximum_price = infosys.tail(90)['Close Price'].max()
minimum_price = infosys.tail(90)['Close Price'].min()
mean_price = infosys.tail(90)['Close Price'].mean()

### Query 1.3
Analyse the data types for each column of the dataframe. Pandas knows how to deal with dates in an intelligent manner. But to make use of Pandas functionality for dates, you need to ensure that the column is of type 'datetime64(ns)'. Change the date column from 'object' type to 'datetime64(ns)' for future convenience. See what happens if you subtract the minimum value of the date column from the maximum value.

In [4]:
infosys['Date'] = infosys.Date.astype('datetime64[ns]')
infosys.Date.max() - infosys.Date.min()

Timedelta('728 days 00:00:00')

### Query 1.4
In a separate array , calculate the monthwise VWAP (Volume Weighted Average Price ) of the stock. 
( VWAP = sum(price*volume)/sum(volume) ) 
To know more about VWAP , visit - VWAP definition 
{Hint : Create a new dataframe column ‘Month’. The values for this column can be derived from the ‘Date” column by using appropriate pandas functions. Similarly, create a column ‘Year’ and initialize it. Then use the 'groupby()' function by month and year. Finally, calculate the vwap value for each month (i.e. for each group created).

In [5]:
infosys['Month'] = infosys.Date.apply(lambda x : x.month)
infosys['Year'] = infosys.Date.apply(lambda x : x.year)
infosys['P*V'] = infosys['Last Price']*infosys['Total Traded Quantity']
temp = infosys.groupby(['Year','Month']).sum()[['P*V','Total Traded Quantity']]
infosys.drop(['Month','Year','P*V'],axis=1,inplace=True)
temp['VWAP'] = temp['P*V']/temp['Total Traded Quantity']

### Query 1.5
Write a function to calculate the average price over the last N days of the stock price data where N is a user defined parameter. Write a second function to calculate the profit/loss percentage over the last N days.
Calculate the average price AND the profit/loss percentages over the course of last -
1 week, 2 weeks, 1 month, 3 months, 6 months and 1 year.
{Note : Profit/Loss percentage between N days is the percentage change between the closing prices of the 2 days }

In [6]:
def avg_price(n):
    return infosys.tail(n)['Close Price'].mean()
def profit_loss_percent(n):
    return (infosys.tail(n).reset_index()['Close Price'][n-1] - infosys.tail(n).reset_index()['Close Price'][0])*100/infosys.tail(n).reset_index()['Close Price'][0]
n = [7,14,31,93,186,365]
for i in n:
    print('Last '+str(i)+' days average and profit/loss%',':',round(avg_price(i),2),round(profit_loss_percent(i),2),sep='\t')

Last 7 days average and profit/loss%	:	720.5	-0.54
Last 14 days average and profit/loss%	:	726.99	-0.19
Last 31 days average and profit/loss%	:	736.5	-1.11
Last 93 days average and profit/loss%	:	726.15	11.75
Last 186 days average and profit/loss%	:	778.66	-47.11
Last 365 days average and profit/loss%	:	966.88	-25.88


### Query 1.6
Add a column 'Day_Perc_Change' where the values are the daily change in percentages i.e. the percentage change between 2 consecutive day's closing prices. Instead of using the basic mathematical formula for computing the same, use 'pct_change()' function provided by Pandas for dataframes. You will note that the first entry of the column will have a ‘Nan’ value. Why does this happen? Either remove the first row, or set the entry to 0 before proceeding.

In [7]:
infosys['Day_Perc_Change'] = infosys['Last Price'].pct_change()
infosys['Day_Perc_Change'] = round(infosys['Day_Perc_Change']*100,2)
infosys.fillna(0,limit=1,inplace=True)

### Query 1.7
Add another column 'Trend' whose values are:
<ul>
    <li>'Slight or No change' for 'Day_Perc_Change' in between -0.5 and 0.5</li>
    <li>'Slight positive' for 'Day_Perc_Change' in between 0.5 and 1</li>
    <li>'Slight negative' for 'Day_Perc_Change' in between -0.5 and -1</li>
    <li>'Positive' for 'Day_Perc_Change' in between 1 and 3</li>
    <li>'Negative' for 'Day_Perc_Change' in between -1 and -3</li>
    <li>'Among top gainers' for 'Day_Perc_Change' in between 3 and 7</li>
    <li>'Among top losers' for 'Day_Perc_Change' in between -3 and -7</li>
    <li>'Bull run' for 'Day_Perc_Change' >7</li>
    <li>'Bear drop' for 'Day_Perc_Change' <-7</li>
</ul>        

In [8]:
def trend(x):
    if x < -7:
        return 'Bear drop'
    elif x < -3:
        return 'Among top losers'
    elif x < -1:
        return 'Negative'
    elif x < -0.5:
        return 'Slight negative'
    elif x < 0.5:
        return 'Slight or No change'
    elif x < 1:
        return 'Slight positive'
    elif x < 3:
        return 'Positive'
    elif x < 7:
        return 'Among top gainers'
    else:
        return 'Bear run'
infosys['Trend'] = infosys.Day_Perc_Change.apply(lambda x : trend(x))

### Query 1.8
Find the average and median values of the column 'Total Traded Quantity' for each of the types of 'Trend'.
{Hint : use 'groupby()' on the 'Trend' column and then calculate the average and median values of the column 'Total Traded Quantity'}

In [9]:
infosys.groupby('Trend').mean()['Total Traded Quantity']
infosys.groupby('Trend').median()['Total Traded Quantity']

Trend
Among top gainers      12042170.5
Among top losers       11894014.5
Bear drop              48786302.0
Negative                5405942.0
Positive                5553948.0
Slight negative         3926982.5
Slight or No change     4050728.0
Slight positive         4648417.0
Name: Total Traded Quantity, dtype: float64

### Query 1.9
SAVE the dataframe with the additional columns computed as a csv file week2.csv. In Module 2, you are going to get familiar with matplotlib, the python module which is used to visualize data.

In [10]:
infosys.to_csv('week2.csv',index=False)